In [0]:

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import os
import argparse
from tqdm import tqdm

In [0]:
class basic():
  def __init__(self):
    self.criterion = nn.CrossEntropyLoss()
    #optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)
    self.train_losses = []
    self.test_losses = []
    self.train_acc = []
    self.test_acc = []
    #use_cuda = torch.cuda.is_available()
    self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # model = self.ResNet(self.BasicBlock, [2,2,2,2]).to(device)
    # optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)
    self.epoch = 1
    


  def data_loader(self):
    transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),])
    transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),])
    trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
    self.train_loader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=0)
    testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
    self.test_loader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=0)
    self.classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
    return self.train_loader,self.test_loader,self.classes

  def train(self,model,device,train_loader,optimizer,epoch):
    model.train()
    pbar = tqdm(train_loader)
    correct = 0
    processed = 0
    for batch_idx, (data, target) in enumerate(pbar):
      # get samples
      data, target = data.to(device), target.to(device)

      # Init
      optimizer.zero_grad()
      # In PyTorch, we need to set the gradients to zero before starting to do backpropragation because PyTorch accumulates the gradients on subsequent backward passes. 
      # Because of this, when you start your training loop, ideally you should zero out the gradients so that you do the parameter update correctly.

      # Predict
      y_pred = model(data)

      # Calculate loss
      loss = self.criterion(y_pred, target)
      self.train_losses.append(loss)

      # Backpropagation
      loss.backward()
      optimizer.step()

      # Update pbar-tqdm
      
      pred = y_pred.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
      correct += pred.eq(target.view_as(pred)).sum().item()
      processed += len(data)

      pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
      self.train_acc.append(100*correct/processed)
  def test(self,model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            data, target = data
            data, target = data.to(device), target.to(device)
            output = model(data)
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
            test_loss += self.criterion(output, target)
            # pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            # correct += pred.eq(target.view_as(pred)).sum().item()
    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))
    with torch.no_grad():
      for data in test_loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

    test_loss /= len(self.test_loader.dataset)
    self.test_losses.append(test_loss)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    print('Accuracy of the network of test images: %d %%' % (
    100 * correct / total))
    self.test_acc.append(100. * correct / len(test_loader.dataset))
    for i in range(10):
      print('Accuracy of %5s : %2d %%' % (classes[i], 100 * class_correct[i] / class_total[i]))

In [0]:
class BasicBlock(nn.Module):
        expansion = 1

        def __init__(self, in_planes, planes, stride=1):
            super(BasicBlock, self).__init__()
            self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
            self.bn1 = nn.BatchNorm2d(planes)
            self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
            self.bn2 = nn.BatchNorm2d(planes)

            self.shortcut = nn.Sequential()
            if stride != 1 or in_planes != self.expansion*planes:
                self.shortcut = nn.Sequential(
                    nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                    nn.BatchNorm2d(self.expansion*planes)
                )

        def forward(self, x):
            out = F.relu(self.bn1(self.conv1(x)))
            out = self.bn2(self.conv2(out))
            out += self.shortcut(x)
            out = F.relu(out)
            return out


In [0]:
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

In [0]:
net = ResNet(BasicBlock, [2,2,2,2])
device =  torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = net.to(device)
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)

In [7]:
q = basic()
train,test,classes = q.data_loader()
for epoch in range(30):
  q.train(model,device,train,optimizer,epoch)
  q.test(model,device,test)

Files already downloaded and verified
Files already downloaded and verified


Loss=0.9605137705802917 Batch_id=390 Accuracy=55.40: 100%|██████████| 391/391 [01:09<00:00,  6.26it/s]
Loss=1.0301302671432495 Batch_id=0 Accuracy=62.50:   0%|          | 1/391 [00:00<01:06,  5.83it/s]


Test set: Average loss: 0.0115, Accuracy: 5883/10000 (58.83%)

Accuracy of the network of test images: 58 %
Accuracy of plane : 75 %
Accuracy of   car : 81 %
Accuracy of  bird : 42 %
Accuracy of   cat : 51 %
Accuracy of  deer : 48 %
Accuracy of   dog : 21 %
Accuracy of  frog : 81 %
Accuracy of horse : 59 %
Accuracy of  ship : 58 %
Accuracy of truck : 68 %


Loss=1.0477604866027832 Batch_id=390 Accuracy=63.13: 100%|██████████| 391/391 [01:09<00:00,  6.36it/s]
Loss=0.9325343370437622 Batch_id=0 Accuracy=67.97:   0%|          | 1/391 [00:00<01:09,  5.61it/s]


Test set: Average loss: 0.0096, Accuracy: 6549/10000 (65.49%)

Accuracy of the network of test images: 65 %
Accuracy of plane : 52 %
Accuracy of   car : 81 %
Accuracy of  bird : 60 %
Accuracy of   cat : 55 %
Accuracy of  deer : 51 %
Accuracy of   dog : 50 %
Accuracy of  frog : 77 %
Accuracy of horse : 75 %
Accuracy of  ship : 79 %
Accuracy of truck : 55 %


...


Test set: Average loss: 0.0042, Accuracy: 8729/10000 (87.29%)

Accuracy of the network of test images: 87 %
Accuracy of plane : 95 %
Accuracy of   car : 100 %
Accuracy of  bird : 73 %
Accuracy of   cat : 72 %
Accuracy of  deer : 85 %
Accuracy of   dog : 90 %
Accuracy of  frog : 90 %
Accuracy of horse : 93 %
Accuracy of  ship : 91 %
Accuracy of truck : 93 %


Loss=0.1685258448123932 Batch_id=390 Accuracy=93.95: 100%|██████████| 391/391 [01:09<00:00,  6.23it/s]
Loss=0.14519855380058289 Batch_id=0 Accuracy=92.19:   0%|          | 1/391 [00:00<01:11,  5.46it/s]


Test set: Average loss: 0.0043, Accuracy: 8777/10000 (87.77%)

Accuracy of the network of test images: 87 %
Accuracy of plane : 95 %
Accuracy of   car : 100 %
Accuracy of  bird : 78 %
Accuracy of   cat : 69 %
Accuracy of  deer : 93 %
Accuracy of   dog : 75 %
Accuracy of  frog : 92 %
Accuracy of horse : 90 %
Accuracy of  ship : 91 %
Accuracy of truck : 96 %


Loss=0.17687879502773285 Batch_id=390 Accuracy=94.05: 100%|██████████| 391/391 [01:08<00:00,  6.40it/s]



Test set: Average loss: 0.0043, Accuracy: 8739/10000 (87.39%)

Accuracy of the network of test images: 87 %
Accuracy of plane : 93 %
Accuracy of   car : 100 %
Accuracy of  bird : 73 %
Accuracy of   cat : 67 %
Accuracy of  deer : 87 %
Accuracy of   dog : 75 %
Accuracy of  frog : 94 %
Accuracy of horse : 93 %
Accuracy of  ship : 91 %
Accuracy of truck : 100 %
